In [4]:
import json
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [63]:
LANDMARKS = [0, 11, 12, 15, 16, 23, 24, 25, 26, 27, 28]

class FallSequenceDataset(Dataset):
    def __init__(self, json_files, sequence_length=24):
        self.sequence_length = sequence_length
        self.sequences = []
        self.labels = []
        self.scaler = StandardScaler()
        
        for json_file in json_files:
            print(f'Processing file: {json_file}')
            with open(json_file, 'r') as f:
                data = json.load(f)
            
            frames = list(data['pose_data'].values())
            
            for i in range(0, len(frames) - self.sequence_length + 1, self.sequence_length):
                sequence = frames[i:i+self.sequence_length]
                if len(sequence) < self.sequence_length:
                    break
                
                landmarks = []
                fall_frames = 0
                for frame in sequence:
                    frame_landmarks = []
                    for landmark in LANDMARKS:
                        frame_landmarks.extend([
                            frame[f'landmark_{landmark}']['x'],
                            frame[f'landmark_{landmark}']['y'],
                            frame[f'landmark_{landmark}']['z']
                        ])
                    landmarks.append(frame_landmarks)
                    if frame['class'] == 'Fall' : 
                        fall_frames += 1
                
                landmarks = np.array(landmarks)
                landmarks = self.scaler.fit_transform(landmarks)
                
                # 레이블 재정의
                if fall_frames == 0:
                    label = 0  # 비낙상
                elif fall_frames == self.sequence_length:
                    label = 2  # 완전 낙상
                else:
                    label = 1  # 낙상 시작
                
                self.sequences.append(landmarks)
                self.labels.append(label)
           
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        return torch.FloatTensor(self.sequences[idx]), torch.LongTensor([self.labels[idx]]).squeeze()

class FallDetectionGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(FallDetectionGRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.gru(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# 데이터 로드 및 모델 학습
json_folder = 'D:\\human_fall\\re_landmark\\re_train_NY_json'
json_files = [os.path.join(json_folder, f) for f in os.listdir(json_folder) if f.endswith('.json')]
dataset = FallSequenceDataset(json_files)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

if len(dataset) > 0:
    sample_sequence, sample_label = dataset[0]
    print(f"Sample sequence shape: {sample_sequence.shape}")
    print(f"Sample label shape: {sample_label.shape}")
    
    input_size = sample_sequence.shape[1]
    print(f"Actual input size: {input_size}")
    hidden_size = 64
    num_layers = 2
    output_size = 3  # 비낙상, 낙상 시작, 완전 낙상
    model = FallDetectionGRU(input_size, hidden_size, num_layers, output_size).to(device)
else:
    print("Dataset is empty")
    exit()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=10, factor=0.5)

num_epochs = 500
best_loss = float('inf')
patience = 30
no_improve = 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    total_acc = 0
    total_batches = 0
    
    for sequences, labels in dataloader:
        print(f"Batch sequences shape: {sequences.shape}")
        print(f"Batch labels shape: {labels.shape}")
        sequences, labels = sequences.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels.view(-1))
        loss.backward()
        optimizer.step()
        
        predicted = torch.argmax(outputs, dim=1)
        acc = (predicted == labels.view(-1)).float().mean()
        
        total_loss += loss.item()
        total_acc += acc.item()
        total_batches += 1
        
        break
    
    avg_loss = total_loss / total_batches
    avg_acc = total_acc / total_batches
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {avg_acc:.4f}')
    
    scheduler.step(avg_loss)
    
    if avg_loss < best_loss:
        best_loss = avg_loss
        no_improve = 0
        torch.save(model.state_dict(), 'best_fall_detection_gru.pt')
    else:
        no_improve += 1
    
    if no_improve >= patience:
        print("Early stopping")
        break

print("Training completed")

Processing file: D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C1.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C2.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C3.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C4.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C5.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C6.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C7.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C8.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00067_H_A_SY_C1.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00067_H_A_SY_C2.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00067_H_A_SY_C3.json
Processing file: D:\human_fall\re_landmark\re_train_NY_json\00067_H_A_SY_C4.json
Processing file: D:\human_fall\re_la

In [45]:
print(f"JSON 파일 수: {len(json_files)}")
for file in json_files[:5]:  # 처음 5개 파일만 출력
    print(file)

JSON 파일 수: 1464
D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C1.json
D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C2.json
D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C3.json
D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C4.json
D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C5.json


In [42]:
def print_json_structure(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    print(f"File: {json_file}")
    print(f"Keys: {data.keys()}")
    if 'pose_data' in data:
        print(f"Number of frames: {len(data['pose_data'])}")
        print(f"Frame keys: {list(data['pose_data'].values())[0].keys()}")
    else:
        print("No 'pose_data' key found")

# 처음 몇 개의 파일만 확인
for json_file in json_files[:3]:
    print_json_structure(json_file)

File: D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C1.json
Keys: dict_keys(['video_path', 'pose_data'])
Number of frames: 24
Frame keys: dict_keys(['landmark_0', 'landmark_11', 'landmark_12', 'landmark_15', 'landmark_16', 'landmark_23', 'landmark_24', 'landmark_25', 'landmark_26', 'landmark_27', 'landmark_28', 'class', 'confidence'])
File: D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C2.json
Keys: dict_keys(['video_path', 'pose_data'])
Number of frames: 25
Frame keys: dict_keys(['landmark_0', 'landmark_11', 'landmark_12', 'landmark_15', 'landmark_16', 'landmark_23', 'landmark_24', 'landmark_25', 'landmark_26', 'landmark_27', 'landmark_28', 'class', 'confidence'])
File: D:\human_fall\re_landmark\re_train_NY_json\00057_H_A_N_C3.json
Keys: dict_keys(['video_path', 'pose_data'])
Number of frames: 25
Frame keys: dict_keys(['landmark_0', 'landmark_11', 'landmark_12', 'landmark_15', 'landmark_16', 'landmark_23', 'landmark_24', 'landmark_25', 'landmark_26', 'landmark_27', '

In [51]:
print(torch.min(labels), torch.max(labels))

tensor(24) tensor(24)
